In [1]:
import mlflow

# Step 2: Set up the MLflow tracking server - EC2
# mlflow.set_tracking_uri("http://ec2-3-81-236-116.compute-1.amazonaws.com:5000/")

# Step 2: Set up the MLflow tracking server - local
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [2]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='mlflow-artifacts:/602507174562575349', creation_time=1733052718653, experiment_id='602507174562575349', last_update_time=1733052718653, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={'mlflow.sharedViewState.18b6912cf74fa902831e9d8dc77772941ea4ec17a69b1246999c67a35c584f16': 'deflate;eJxtksFu2zAMhl9l0DkY0h59c4N0K5YMg9MFA4IhVSQmJkBLhih18Yq8+ygnnd0gR378f4oU+aYYdDD1I1KEoAqlJsoHC+Gh+wadxDrGgLsUgT9z1CFuIzYwiEo2qthrYpioPv+c04UqFwsREe7BdIbgf/nSRHzNfqujZoj8ntn8nqjGW6A1BEbvBgfRpyo5Fg8DgYlgZ55SI6TYjNt7WfkUDLyIcEyXuShn2oBQI2i6DWA00Qd4dwve34LamBS06a5qtiLF3PpV2dv8fsxl9iAjzo+tdhasKt5OZ/IDncvx5qL4itaCG+K1FNghYeyWuh1sZ1meXD7w8alaPW/vptvq5/eVtPCK8Gepj9jg31z6sjyxLZDj+wMXilwaI6uWLivoVz6yJIYvwacW7FpTAn5ys1ouQPYSQ5J8jRbmTRu7j/iQPQ9yWy4RXcKr0XWKvoJ9AK7nTu9o9OiB/E7TAh30VWfe7fEgLnUsj8jnm+UIbX99DlaN97F2wPL8dLigX1m87HdxtqjT6R85zBYB'}>

In [3]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, ADASYN
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

c:\Data-Science\CampusX\Projects\youtube-comment-analyzer\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv("dataset.csv").dropna(subset=["clean_comment"])
df.shape

(36662, 2)

In [5]:
#  Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df["category"] = df["category"].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=["category"])

ngram_range = (1, 3)  # Trigram setting
# ngram_range = (1, 2)  # Bigram setting

max_features = 1000  # Set max_features to 1000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_comment"], df["category"], test_size=0.2, random_state=42, stratify=df["category"]
)

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

# smote = SMOTE(random_state=42)
adasyn = ADASYN(random_state=42)

# X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vec, y_train)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_vec, y_train)

In [6]:
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", f"{model_name}_triADASYSN")

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 10)

    model = XGBClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42,
    )
    return accuracy_score(y_test, model.fit(X_train_resampled, y_train_resampled).predict(X_test_vec))


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(
        n_estimators=best_params["n_estimators"],
        learning_rate=best_params["learning_rate"],
        max_depth=best_params["max_depth"],
        random_state=42,
    )

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_resampled, X_test_vec, y_train_resampled, y_test)


# Run the experiment for XGBoost
run_optuna_experiment()

[I 2024-12-02 11:12:58,399] A new study created in memory with name: no-name-326a12ea-a4f2-4a66-b1ee-2ccc7d9526c8
[I 2024-12-02 11:13:28,782] Trial 0 finished with value: 0.6335742533751534 and parameters: {'n_estimators': 154, 'learning_rate': 0.008011975231730929, 'max_depth': 6}. Best is trial 0 with value: 0.6335742533751534.
[I 2024-12-02 11:13:42,998] Trial 1 finished with value: 0.6862130096822583 and parameters: {'n_estimators': 257, 'learning_rate': 0.03574762075492366, 'max_depth': 3}. Best is trial 1 with value: 0.6862130096822583.
[I 2024-12-02 11:15:59,261] Trial 2 finished with value: 0.6282558298104459 and parameters: {'n_estimators': 230, 'learning_rate': 0.0010996708501398197, 'max_depth': 10}. Best is trial 1 with value: 0.6862130096822583.
[I 2024-12-02 11:16:42,052] Trial 3 finished with value: 0.6758489022228283 and parameters: {'n_estimators': 197, 'learning_rate': 0.018744678793224143, 'max_depth': 6}. Best is trial 1 with value: 0.6862130096822583.
[I 2024-12-02

🏃 View run XGBoost_ADASYN_TFIDF_Trigrams at: http://127.0.0.1:5000/#/experiments/602507174562575349/runs/8fccef9b329244049ed4aefffb5d20e9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/602507174562575349


### Best parameters for XGBOOST

`SMOTE with Trigram setting`

` {'n_estimators': 298, 'learning_rate': 0.09929217871014055, 'max_depth': 9}. Best is trial 27 with value: 0.9035720570121415`